In [ ]:
import pandas as pd
import datetime
from alice_blue import *

In [ ]:
#Get an access token
access_token = AliceBlue.login_and_get_access_token(username='XXXX', password='XXX', 
                                                    twoFA='act',
                                                    api_secret=
                                                    'XXXXX')
#Create AliceBlue Object, once you have your access_token
#Get master contracts
alice = AliceBlue(username='XXXX', password='XXXX', access_token=access_token,
                  master_contracts_to_download=['NSE','NFO','BSE'])

In [ ]:
ltp = 0
socket_opened = False

def event_handler_quote_update(message):
    global ltp
    ltp = message['ltp']

def open_callback():
    global socket_opened
    socket_opened = True

In [ ]:
socket_opened = False
alice.start_websocket(subscribe_callback=event_handler_quote_update,
                      socket_open_callback=open_callback,
                      run_in_background=True)
while(socket_opened==False):    # wait till socket open & then subscribe
    pass

In [ ]:
distance_from_strike = int(input("Please enter the distance value from strike"))
underlying_instrument = str(input("Please enter instrument symbol or token number of instrument (enter in Capitals)"))
instrument_expiry_date = str(input("please enter instruement expiry date"))

In [ ]:
def get_nse_ltp(ins_token):
    ins_script = alice.get_instrument_by_token('NSE',ins_token)
    alice.subscribe(ins_script, LiveFeedType.COMPACT)
    alice.unsubscribe(ins_script,LiveFeedType.COMPACT)
    return ltp

def get_nfo_ltp(ins_symbol):
    ins_script = alice.get_instrument_by_symbol('NFO',ins_symbol)
    alice.subscribe(ins_script, LiveFeedType.COMPACT)
    alice.unsubscribe(ins_script,LiveFeedType.COMPACT)
    return ltp

In [ ]:
OTMCall = underlying_instrument_value + 300
OTMPut = underlying_instrument_value - 300

OTMCall_ins_symbol = underlying_instrument + ' ' + instrument_expiry_date + ' ' + str(OTMCall) +' CE'
OTMPut_ins_symbol = underlying_instrument + ' ' + instrument_expiry_date + ' ' + str(OTMPut) +' PE'

In [ ]:
ins_script = alice.get_instrument_by_symbol('NFO',OTMPut_ins_symbol)
alice.subscribe(ins_script, LiveFeedType.COMPACT)
alice.unsubscribe(ins_script,LiveFeedType.COMPACT)

In [ ]:
nifty_ltp = get_nse_ltp(26000)

In [ ]:
one_minute_ltp = []

previous_OTMCall = 0.0
previous_OTMPut = 0.0
count = 0
while(True):
    if datetime.datetime.today().second % 5 == 0:
        if len(one_minute_ltp) <= 10:
            one_minute_ltp.append(nifty_ltp)
        else:
            one_minute_ltp.pop()
    
    if len(one_minute_ltp) > 4:       
        underlying_instrument_ltp = (one_minute_ltp[0] + one_minute_ltp[1] + one_minute_ltp[2]) / 3

        rem = underlying_instrument_ltp % 50
        if rem % 50 <= 25:
            underlying_instrument_value = underlying_instrument_ltp - rem
        else:
            underlying_instrument_value = underlying_instrument_ltp - rem + 50

        OTMCall = underlying_instrument_value + 300
        OTMPut = underlying_instrument_value - 300
        
        OTMCall_ins_symbol = underlying_instrument + ' ' + instrument_expiry_date + ' ' + str(OTMCall) +' CE'
        OTMPut_ins_symbol = underlying_instrument + ' ' + instrument_expiry_date + ' ' + str(OTMPut) +' PE'
        
        otmcall_ltp = get_nfo_ltp(OTMCall_ins_symbol)
        
        otmput_ltp = get_nfo_ltp(OTMPut_ins_symbol)
        
        hour = datetime.datetime.today().hour
        minute = datetime.datetime.today().minute

        if ((hour == 17  and minute == 59) and (count == 0)):
            print(datetime.datetime.today(),"OTMCall :" +str(OTMCall),"OTMPut :" +str(OTMPut),"LTP : " + str(nifty_ltp),
                  "otmcall_ltp : " +str(otmcall_ltp),"otmput_ltp : " +str(otmput_ltp),"Sell signal")
            previous_OTMCall = OTMCall
            previous_OTMPut = OTMPut
            count = count + 1

        elif(((OTMCall != previous_OTMCall) or (OTMPut != previous_OTMPut)) and (count == 1)) :

            previous_OTMCall = OTMCall
            previous_OTMPut = OTMPut
            print(datetime.datetime.today(),"OTMCall :" +str(OTMCall),"OTMPut :" +str(OTMPut),"LTP : " + str(nifty_ltp),
                  "otmcall_ltp : " +str(otmcall_ltp),"otmput_ltp : " +str(otmput_ltp),"buy_signal on prevoius OTMS")
            print(datetime.datetime.today(),"OTMCall :" +str(OTMCall),"OTMPut :" +str(OTMPut),"LTP : " + str(nifty_ltp),
                  "otmcall_ltp : " +str(otmcall_ltp),"otmput_ltp : " +str(otmput_ltp),"sell signal on current OTMS")
    if datetime.datetime.today().minute == 0:
        break